In [33]:
import csv 
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import MinMaxScaler

# kNN

In [2]:
def kNN(data, attribute, labels, k):
    ary = sp.spatial.distance.cdist(data, attribute).flatten()
    now_sorted=ary.argsort()
    k_nearest=np.zeros(k)
    for i in range(k):
        k_nearest[i]=now_sorted[i]
    counter=0
    for i in range(k):
        ind=int(k_nearest[0])
        ind_next=int(k_nearest[i])
        if labels[ind]==labels[ind_next]:
            counter+=1
        else:
            res=labels[ind_next]
    if counter>k/2:
        result_label=labels[ind]
    else:
        result_label=res
    return result_label

In [3]:
def LOO_kNN(data, labels, k):
    labels_1=pd.DataFrame(labels)
    data_1=pd.DataFrame(data)
    counter=0
    data_copy=pd.DataFrame(data)
    for i in range(data_copy.shape[0]):
        rest_data=data_1.drop(data_1.index[i]).values
        rest_data_labels=labels_1.drop(labels_1.index[i]).values
        test=data_copy.iloc[[i]].values
        res=kNN(rest_data, test, rest_data_labels,k)
        if res[0]!=labels_1.iloc[[i]].values[0][0]:
            counter+=1
    res_LOO=counter/(data.shape[0])
    return res_LOO

In [4]:
def choosing_k(data, labels):
    loo_values=np.zeros(10)
    i=0
    for k in range(1, 11):
        print("k=", k)
        loo_values[i]=LOO_kNN(data, labels, k)
        i+=1
        
    print("loo_values:", loo_values)
    sorted_loo=loo_values.argsort() 
    best_k=sorted_loo[0]+1
    return best_k

In [5]:
def accuracy(result, real):
    score=0
    for i in range(result.shape[0]):
        if(result[i]==real[i]): 
            score+=1
    return score/result.shape[0]

# Radius Neighbours

In [6]:
def RadN(data, attribute, labels, radius):
    ary = sp.spatial.distance.cdist(data, attribute).flatten()
    index=[]
    for i in range(ary.shape[0]):
        if ary[i]<radius:
            index.append(i)
    index=np.array(index)
    counter=0;
    if index.shape[0]>0:
        for i in range(index.shape[0]):
            ind=int(index[0])
            ind_next=int(index[i])
            if labels[ind]==labels[ind_next]:
                counter+=1
            else:
                res=labels[ind_next] 
        if counter>index.shape[0]/2:
            result_label=labels[ind]
        else:
            result_label=res
    if index.shape[0]==0:
        result_label=888
    return result_label

In [7]:
def LOO_RN(data, labels, r):
    labels_1=pd.DataFrame(labels)
    data_1=pd.DataFrame(data)
    counter=0
    data_copy=pd.DataFrame(data)
    for i in range(data_copy.shape[0]):
        rest_data=data_1.drop(data_1.index[i]).values
        rest_data_labels=labels_1.drop(labels_1.index[i]).values
        test=data_copy.iloc[[i]].values
        res=RadN(rest_data, test, rest_data_labels,r)
        if res!=888:
            if res[0]!=labels_1.iloc[[i]].values[0][0]:
                counter+=1
        else:
            counter+=1
    res_LOO=counter/(data.shape[0])
    return res_LOO

In [8]:
def choose_r(data, labels):
    ary = sp.spatial.distance.cdist(data, data).flatten()
    max_r=np.amax(ary)
    loo_1=0
    loo_2=8000
    prev_r=max_r
    while(loo_2>0.3):
        current_r=prev_r/3
        print(int(current_r))
        loo_1=LOO_RN(data, labels, prev_r)
        loo_2=LOO_RN(data, labels, current_r)
        if loo_1<0.2:
            break
        prev_r=current_r
        print(loo_2)
    return prev_r

# Spam

In [9]:
data_open = pd.read_csv("spam.csv")
data = data_open.loc [1:10000] 
x = data[data.columns.difference(['label'])].values
y = data.label.values
y_1=pd.DataFrame(y)
x_1=pd.DataFrame(x)

In [10]:
best_k=choosing_k(x, y )
best_r=choose_r(x,y)
x_copy=pd.DataFrame(x)
result_labels=[]
result_labels_rad=[]
for i in range(x_copy.shape[0]):
    data=x_1.drop(x_1.index[i]).values
    data_labels=y_1.drop(y_1.index[i]).values
    test=x_copy.iloc[[i]].values
    result_labels.append(kNN(data, test, data_labels, best_k))
    result_labels_rad.append(RadN(data, test, data_labels, best_r))

k= 1
k= 2
k= 3
k= 4
k= 5
k= 6
k= 7
k= 8
k= 9
k= 10
loo_values: [0.16934783 0.22347826 0.18565217 0.21347826 0.18565217 0.21673913
 0.19586957 0.21673913 0.2026087  0.21978261]
5280
0.39369565217391306
1760
0.38065217391304346
586
0.34152173913043476
195
0.3184782608695652
65
0.2841304347826087


# knn

In [11]:
res=np.array(result_labels).flatten()
acc=accuracy(res, y)

In [12]:
print(f'k={best_k}, accuracy={acc}')

k=1, accuracy=0.8306521739130435


# RN

In [13]:
res_1=np.array(result_labels_rad).flatten()
acc_1=accuracy(res_1, y)

In [14]:
acc_1

0.7158695652173913

In [15]:
print(f'r={best_r}, accuracy={acc_1}')

r=65.18525063208538, accuracy=0.7158695652173913


# Cancer

In [16]:
data_open = pd.read_csv("cancer.csv")
data = data_open.loc [1:10000] 
x = data[data.columns.difference(['label'])].values
y = data.label.values
y_1=pd.DataFrame(y)
x_1=pd.DataFrame(x)

In [17]:
best_k=choosing_k(x, y )
best_r=choose_r(x,y)
x_copy=pd.DataFrame(x)
result_labels=[]
result_labels_rad=[]
for i in range(x_copy.shape[0]):
    data=x_1.drop(x_1.index[i]).values
    data_labels=y_1.drop(y_1.index[i]).values
    test=x_copy.iloc[[i]].values
    result_labels.append(kNN(data, test, data_labels, best_k))
    result_labels_rad.append(RadN(data, test, data_labels, best_r))

k= 1
k= 2
k= 3
k= 4
k= 5
k= 6
k= 7
k= 8
k= 9
k= 10
loo_values: [0.08450704 0.08450704 0.07394366 0.0721831  0.06690141 0.06866197
 0.06866197 0.06690141 0.06690141 0.06514085]
1579
0.3045774647887324
526
0.12852112676056338


# knn

In [18]:
res=np.array(result_labels).flatten()
acc=accuracy(res, y)

In [19]:
print(f'k={best_k}, accuracy={acc}')

k=10, accuracy=0.9348591549295775


# RN

In [20]:
res_1=np.array(result_labels_rad).flatten()
acc_1=accuracy(res_1, y)

In [21]:
acc_1

0.8714788732394366

In [22]:
print(f'r={best_r}, accuracy={acc_1}')

r=526.5654228607513, accuracy=0.8714788732394366


# Normilized Spam

In [23]:
scaler = MinMaxScaler()
data_open = pd.read_csv("spam.csv")
data = data_open.loc [1:10000] 
x = data[data.columns.difference(['label'])].values
x_scaled=scaler.fit_transform(x)
y = data.label.values
y_1=pd.DataFrame(y)
x_1=pd.DataFrame(x_scaled)

In [24]:
best_k=choosing_k(x_scaled, y )
best_r=choose_r(x_scaled,y)
x_copy=pd.DataFrame(x_scaled)
result_labels=[]
result_labels_rad=[]
for i in range(x_copy.shape[0]):
    data=x_1.drop(x_1.index[i]).values
    data_labels=y_1.drop(y_1.index[i]).values
    test=x_copy.iloc[[i]].values
    result_labels.append(kNN(data, test, data_labels, 1))
    result_labels_rad.append(RadN(data, test, data_labels, best_r))

k= 1
k= 2
k= 3
k= 4
k= 5
k= 6
k= 7
k= 8
k= 9
k= 10
loo_values: [0.0876087  0.11826087 0.095      0.115      0.09521739 0.1123913
 0.09934783 0.11108696 0.10195652 0.11130435]
0
0.3932608695652174
0
0.22130434782608696


# kNN

In [25]:
res=np.array(result_labels).flatten()
acc=accuracy(res, y)
print(f'k={best_k}, accuracy={acc}')

k=1, accuracy=0.9123913043478261


# RN

In [26]:
res_1=np.array(result_labels_rad).flatten()
acc_1=accuracy(res_1, y)

In [27]:
print(f'r={best_r}, accuracy={acc_1}')

r=0.31052243563363074, accuracy=0.778695652173913


# Normilized Cancer

In [28]:
scaler = MinMaxScaler()
data_open = pd.read_csv("cancer.csv")
data = data_open.loc [1:10000] 
x = data[data.columns.difference(['label'])].values
x_scaled=scaler.fit_transform(x)
y = data.label.values
y_1=pd.DataFrame(y)
x_1=pd.DataFrame(x_scaled)

In [29]:
best_k=choosing_k(x_scaled, y )
best_r=choose_r(x_scaled,y)
x_copy=pd.DataFrame(x_scaled)
result_labels=[]
result_labels_rad=[]
for i in range(x_copy.shape[0]):
    data=x_1.drop(x_1.index[i]).values
    data_labels=y_1.drop(y_1.index[i]).values
    test=x_copy.iloc[[i]].values
    result_labels.append(kNN(data, test, data_labels, 1))
    result_labels_rad.append(RadN(data, test, data_labels, best_r))

k= 1
k= 2
k= 3
k= 4
k= 5
k= 6
k= 7
k= 8
k= 9
k= 10
loo_values: [0.04753521 0.04225352 0.02992958 0.03521127 0.0334507  0.03169014
 0.02992958 0.02992958 0.02992958 0.03169014]
1
0.18661971830985916


# kNN

In [30]:
res=np.array(result_labels).flatten()
acc=accuracy(res, y)
print(f'k={best_k}, accuracy={acc}')

k=3, accuracy=0.9524647887323944


# RN

In [31]:
res_1=np.array(result_labels_rad).flatten()
acc_1=accuracy(res_1, y)
print(f'r={best_r}, accuracy={acc_1}')

r=1.1547379262300324, accuracy=0.8133802816901409
